<a href="https://colab.research.google.com/github/yohann0412/Math103Agent/blob/main/MathHomeworkAIAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
SYSTEM_PROMPT = '''


'''

In [ ]:
PROOF_ATTEMPT_PROMPT = ''' You are now writing a full proof of the abstract algebra problem using the chosen strategy. Your job is only to write the proof, clearly and rigorously.

Follow these constraints:
- Be explicit in your reasoning.
- Do not skip steps.
- Assume the reader knows abstract algebra, but write with clarity.
- avoid circular reasoning or any logical fallacies

Do not reflect on the quality of the proof. Do not critique it. Just output the best version of the proof you can.



'''

In [ ]:
BREAKDOWN_PROMPT = '''
You are an abstract algebra problem solver. Your only task is to break down the problem into clearly defined parts.

Step 1: Categorize the problem — what type of problem is it? (e.g. proving irreducibility, finding ideals, checking if something is a PID, etc.)

Step 2: Preparse — what are the key objects involved (rings, elements, ideals, etc.)? What definitions or theorems might be relevant?

Step 3: Rephrase the goal in your own words, making it crystal clear what must be proven.

Only do the above three things. Do not suggest ways to solve it. Do not do any algebra.

The problem is:

'''

In [ ]:
IDENTIFY_AND_RANK_PROMPT = '''

You are continuing from a prior breakdown of an abstract algebra problem. Your task is to identify **distinct** ways to attempt a solution — no more than 3.

For each strategy:
- Give a 1-line name for the method.
- Give a 1–2 sentence explanation of how the method would be used to attempt the proof.

Then, rank the strategies from most promising to least promising. Use criteria such as alignment with the problem type, likelihood of success, and simplicity.

Do not perform any steps of the proof. Only identify and rank strategies.

I will send the problem first, followed by the breakdown.

Based on your reasoning, return the best proof strategy, with a brief, clear explanation of how to do it.
'''

In [ ]:
CRITIQUE_PROMPT = '''
You are now evaluating a proof of an abstract algebra problem. Your task is only to critique it.

Check for:
- Logical errors
- Gaps or missing justifications
- Use of incorrect definitions
- Any unclear or imprecise statements

At the end, answer these questions:
- Is anything majorly wrong with this proof? (Yes/No)
- If yes, should we attempt a different proof strategy instead?

If after your analysis, the answer to both questions is no, explicitly say "there is nothing wrong with this" and nothing else.
Do not propose fixes. Just critique.

'''

# How do you solve a math equation?

1. CATEGORIZE IT INTO A TYPE OF PROBLEM
2. PREPARSE THE PROBLEM - figure out what the 'objects' are, what we're broadly dealing with.
Once the category of the problem is clear, and we have an idea of the tools we are dealing with:
- break down steps to prove the question
- understand all fundamental properties of the object(s) we are dealing with
- proceed in logical steps, ensuring each step builds on the previous, with no skips or fallacies
- verify that the goal was achieved and that all assumptions were used or acknowledged

## Agent properties:
- System prompt
- messages
- breakdown of question, including understanding whats going on
- the actual solution
- the edge case detector / rigor detector
- reviewing the math and rewriting it. repeating this till we have a solution that is satisfactory.

**First node** -  breakdown the question\
**Second Node** - start thinking of some ways to do it\
**Third node** - rank the ways to solve it\
**Fourth node** - write out the proof to solve it\
**Fifth node** - critique the proof, is there anything wrong with it? rigor, clarity, communication etc.\
send it back to the previous node, till we either reach the turn limit or its saying theres no proper issues. Ask it if theres something majorly wrong, and if so, we can use another proof technique.
Return.


In [ ]:
from openai import OpenAI
import os
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
client = OpenAI()



def gptHelperFunction(systemPrompt, modelName, userPrompt):
  completion = client.chat.completions.create(
    model=modelName,
    messages=[
        {"role": "system", "content": systemPrompt},
        {
            "role": "user",
            "content": userPrompt,
        },
    ],
)
  return completion.choices[0].message.content

In [ ]:
class MathAgent:


  def __init__(self, systemPrompt):
    self.systemPrompt = systemPrompt
    self.messages = []
    self.breakdown = ""
    self.rankingOfSolutions = ""
    self.proofAttempts = []
    self.critiques = []
    self.messages.append(systemPrompt)

  def breakingDownProblem(self, mathQuestion):
    BREAKDOWN_PROMPT += mathQuestion
    questionBreakdown = gptHelperFunction(BREAKDOWN_PROMPT, "gpt-4.1", mathQuestion)
    return questionBreakdown

  def identifyAndRankSolutions(self, mathQuestion, questionBreakdown):
    optimalSolutionMethod = gptHelperFunction(IDENTIFY_AND_RANK_PROMPT, "gpt-3.5-turbo",
                                 f"The question is: {mathQuestion}\nand the breakdown is: {questionBreakdown}")
    return optimalSolutionMethod

  def attemptingProof(self, mathQuestion, questionBreakdown, solution):
    proofAttempts = gptHelperFunction(PROOF_ATTEMPT_PROMPT, "gpt-3.5-turbo", )
    return proofAttempts

  def critique(self, mathQuestion, proofAttempts):
    critiques = gptHelperFunction(CRITIQUE_PROMPT, "gpt-3.5-turbo", )
    return critiques

  def __call__(self, mathQuestion):
    self.breakdown = breakingDownProblem(mathQuestion)

    self.rankingOfSolutions = identifyAndRankSolutions(mathQuestion, self.breakdown)

    self.proofAttempts += attemptingProof(mathQuestion, self.breakdown, self.rankingOfSolutions[0], self.critiques)

    self.critiques = critique(mathQuestion, self.proofAttempts)

    while self.critiques:

      self.proofAttempts += attemptingProof(mathQuestion, self.breakdown, self.rankingOfSolutions[0])
      self.critiques = critique(mathQuestion, self.proofAttempts[-1])

    return self.proofAttempts






